In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.upgrade_sdc import UpgradeSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TrueCoffee'

corpus, labels = load_corpus('../data/' + file_name + '.txt')

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 350
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 948 words
filter frequent words: 370 words
filter letter words: 368 words
filter stop words: 219 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [6]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [7]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [9]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [10]:
min_samples = 7
eps = 0.32
epoch = 15

onehot_corpus = get_onehot(idx_corpus, 'normal')
# onehot_corpus = get_onehot(filtered_corpus, 'tfidf')

max_compactness = 0
for i in range(epoch):
#     model = UpgradeSDC()
#     _tpredicted_labels, _tmarks = model.predict(onehot_corpus, min_samples, eps)

    model = SDC()
    _tpredicted_labels, _tmarks = model.predict(onehot_corpus, min_samples, eps)
    
#     _tmarks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=7).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, _tmarks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)
num_cluster = label_count[0][-1] + 1
print(label_count, '\n')

193.99510414588792
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]), array([58, 15, 44, 35, 15, 10,  7, 10, 18, 20, 30, 24, 12, 14, 14, 16,  8])) 



In [11]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
9 6 0.3076748484211804
[0, 1, 2, 3, 4, 5, 6, 7, 8, 6, 10, 11, 12, 13, 14, 15, 16]


### Result

In [12]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [13]:
# result.to_csv('../data/results/2/' + file_name + '.csv')

# result = pandas.read_csv('../data/results/2/' + file_name + '.csv')

count = 0
for index, value in result.iterrows():
    if value['marks'] == -1:
        count += 1
print(count)

90


In [14]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16), value=0)

1 เรางง สั่งคาปูชิโนหวานน้อย ทำไม่ได้ค่ะ ถ้าลูกค้าอยากได้หวานน้อยต้องสั่งลาเต้ กูงงงงงงง
4 ซื้อลิขสิทธิ์ UCL มาด้วยนะ จ่ายรายเดือนไป แต่ไม่มีให้ดู
21 ทำไมเอาเมนูทวิซที่ปั่นใส่ขนมช็อกโกแลตทวิซคาลาเมลออก 💥🔥🔥🔥โกรธหนักค่ะ เพราะชอบมากกกกกกกก
24 ขนาดตอนเป็นลูกค้าทรูมูฟ ยังไม่กิน ตอนนี้ย้ายค่ายแล้ว อย่าหวังจะได้แดก
27 เหมือนจะพรีเมียม แต่เมื่อลองแร้ว เอิ่มมม บางตัวโอเครนะ ค่อยๆปรับไปทีละจุด
28 ขอทำขนมไปส่งได้ไหมคะ นักเรียนบ้านยากจนจะสอนนักเรียนแต่ไม่มีที่ให้โอกาสนักเรียนคะ แต่รับรองขนมอร่อยคะ
34 โปรแต้มมรึงหมดเร็วไปนะ เหมือน limit net มือถือ ... 5 วันหมดอิดอก
42 ตัวอักษรใหญ่ๆ มองเมนูไม่ชัด ไม่กล้าสั่ง กลัวสั่งผิดละเขินน 555+
57 เป็นร้านที่แดกกาแฟร้อนทีไร แม่งต้องล้วกลิ้นไหม้เป็นแผลตลอด น้ำร้อนเอ็งจะร้อนไปไหนวะ สงสัยแม่งเอามาจากบ่อในนรกชัวร์ๆ ถ้าเอ็งร้อนแบบนี้
70 Iced Black Tea Latte หาชงอร่อยยากมาก อยากกินทีต้องลุ้นทีว่ามือชงจะทำได้แค่ไหน เข้าร้านทรูเพื่อการนี้โดยเฉพาะ เมื่อไม่เจอมือเซียนชง ก็ไม่เข้าไปเหยียบร้านค่ะ
83 ใช้ค่ายโทรศัพ์ ใช้ wifi โปรแบบแพงๆ มีสิทแบคการ์ดถือไว้หลายปีสิทธ์เต็มตลอดทั

In [15]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), value=0)

1 เรางง สั่งคาปูชิโนหวานน้อย ทำไม่ได้ค่ะ ถ้าลูกค้าอยากได้หวานน้อยต้องสั่งลาเต้ กูงงงงงงง
4 ซื้อลิขสิทธิ์ UCL มาด้วยนะ จ่ายรายเดือนไป แต่ไม่มีให้ดู
21 ทำไมเอาเมนูทวิซที่ปั่นใส่ขนมช็อกโกแลตทวิซคาลาเมลออก 💥🔥🔥🔥โกรธหนักค่ะ เพราะชอบมากกกกกกกก
24 ขนาดตอนเป็นลูกค้าทรูมูฟ ยังไม่กิน ตอนนี้ย้ายค่ายแล้ว อย่าหวังจะได้แดก
27 เหมือนจะพรีเมียม แต่เมื่อลองแร้ว เอิ่มมม บางตัวโอเครนะ ค่อยๆปรับไปทีละจุด
28 ขอทำขนมไปส่งได้ไหมคะ นักเรียนบ้านยากจนจะสอนนักเรียนแต่ไม่มีที่ให้โอกาสนักเรียนคะ แต่รับรองขนมอร่อยคะ
34 โปรแต้มมรึงหมดเร็วไปนะ เหมือน limit net มือถือ ... 5 วันหมดอิดอก
42 ตัวอักษรใหญ่ๆ มองเมนูไม่ชัด ไม่กล้าสั่ง กลัวสั่งผิดละเขินน 555+
57 เป็นร้านที่แดกกาแฟร้อนทีไร แม่งต้องล้วกลิ้นไหม้เป็นแผลตลอด น้ำร้อนเอ็งจะร้อนไปไหนวะ สงสัยแม่งเอามาจากบ่อในนรกชัวร์ๆ ถ้าเอ็งร้อนแบบนี้
70 Iced Black Tea Latte หาชงอร่อยยากมาก อยากกินทีต้องลุ้นทีว่ามือชงจะทำได้แค่ไหน เข้าร้านทรูเพื่อการนี้โดยเฉพาะ เมื่อไม่เจอมือเซียนชง ก็ไม่เข้าไปเหยียบร้านค่ะ
83 ใช้ค่ายโทรศัพ์ ใช้ wifi โปรแบบแพงๆ มีสิทแบคการ์ดถือไว้หลายปีสิทธ์เต็มตลอดทั

In [16]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), value=0)

1 ['ลูกค้า', 'ทำ', 'หวาน', 'สั่ง', 'สั่ง']
4 ['ซื้อ', 'ดู', 'จ่าย', 'เดือน']
21 ['ชอบ', 'เมนู', 'ขนม']
24 ['กิน', 'ลูกค้า', 'ตอน', 'ตอน', 'ขนาด']
27 ['เหมือน', 'ตัว', 'เอิ่ม']
28 ['อร่อย', 'ทำ', 'ไหม', 'เรียน', 'เรียน']
34 ['เหมือน', 'มือถือ', 'อิดอก']
42 ['สั่ง', 'สั่ง', 'เมนู', 'กล้า', 'ตัว']
57 ['กาแฟ', 'ร้าน', 'น้ำ', 'แดก', 'แม่ง']
70 ['อร่อย', 'ร้าน', 'ชง', 'ชง', 'ทำ']
83 ['ราคา', 'แพง', 'เต็ม', 'ลด', 'เรื่อง']
91 ['กาแฟ', 'น้ำ', 'สาขา', 'เต็ม', 'แก้ว']
93 ['เนต', 'เนต', 'ขนาด', 'ตัด', 'หมุน']
98 ['หวาน']
104 ['ทำ', 'โปรเยอะ']
105 []
107 []
112 ['สัญญาณ', 'เล่น']
117 ['รส', 'ชาติ']
118 []
121 []
125 ['จืด']
126 ['พนง']
131 ['แดก']
134 []
146 ['จาง']
150 ['โปร', 'ดดดด', 'เหอะ']
151 ['ทาน']
161 []
169 ['แดก']
179 []
185 ['รสชาด']
187 ['ซื้อ', 'แถม', 'มากกกก']
191 ['ขาย', 'ผม', 'สำหรับ', 'ปี']
199 ['รสชาด']
203 ['ซื้อ', 'ทิ้ง']
206 ['เย็น', 'นม', 'ใส่', 'ลาเต้', 'ขม']
226 []
235 ['ทิ้ง', 'ตัด']
240 ['ไหร่']
251 ['เหมือน', 'บัตร', 'แบล็คการ์ด']
255 ['บ้าน', 'กาก', 'หมุน']
257 ['รส']
2

In [17]:
seed = 211
compare = 53

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(corpus[seed])
print(filtered_corpus[compare])
print(corpus[compare])
# print(sims[29])

[[0.37796447]]
['พนักงาน']
พนักงาน
['เต็ม', 'พนักงาน', 'แลก', 'ลอง', 'แต้ม']
ลองใช้แต้มแลกสิ พนักงานจะบอกว่าเต็มแล้วแล้วจะออกโปรโมชั่นมาเพื่ออะไร
